In [ ]:
import sys
sys.path.append('../../pyclk')
sys.path.append('../bundle')

import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
from dask.dot import dot_graph
import fpga_op
from fpga import FPGA
from pyclk import Trace

%matplotlib inline

In [ ]:
iter_nb = 2
func_nb = 3

fpga = FPGA(iter_nb, func_nb)

fpga_op.fpga = fpga

trace = Trace()
trace.add(fpga.u_crossbar.r_iter_i)
trace.add(fpga.u_crossbar.r_func_i)
for i in range(iter_nb):
    trace.add(fpga.u_iterator[i].o_op_done)
    trace.add(fpga.u_crossbar.r_iter_func_i[i])
    trace.add(fpga.u_crossbar.r_iter_busy[i])
for i in range(func_nb):
    trace.add(fpga.u_func[i].o_res)
    trace.add(fpga.u_crossbar.r_func_busy[i])

fpga.trace = trace

In [ ]:
size = 1000
chunks = 111

x = da.arange(0, size, chunks=(chunks,))
y = da.ones(size, chunks=(chunks,))
z = fpga_op.add(x, y)
#dot_graph(z.dask)

x_true = np.arange(0, size)
y_true = np.ones(size)
z_true = x_true + y_true

In [ ]:
z = z.compute()
if np.array_equal(z, z_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
trace.plot()

In [ ]:
diff_idx = np.where(z != z_true)
plt.figure(figsize=(15, 5))
plt.scatter(diff_idx, z[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, z_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
plt.xlim(0, size)
plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
z[diff_idx]

In [ ]:
z_true[diff_idx]